In [2]:
import uuid
from flask import Flask, request, jsonify
from utilities import logger
from flask_cors import CORS
import sys
from xinference.client import Client

mylogger = logger.Logger(name='llmtool', debug=True).logger  
# app = Flask(__name__)  
# CORS(app)

# 使用字典模拟用户存储，实际项目中应使用数据库
users_db = {
    "admin": {"password": "123456"},
    'zz': {'password': 'zz'},
    'xyz': {'password': 'abc'}
}

# 存储登录成功的用户session数据
sessions = {}



In [ ]:
x_addr = 'http://10.101.9.50:9998'
mylogger.debug(f"Start to connect xinference server at {x_addr}")
try:
    client = Client(x_addr)    
    model_uid = client.launch_model(model_name="bge-large-zh-v1.5", model_type="embedding")    
    model = client.get_model(model_uid)
except Exception as e:
    mylogger.error(f"Something goes wrong: {e}")   
def create_embedding_bge(sentence):
    result = model.create_embedding(sentence)
    embedding = result['data'][0]['embedding']
    # LOG.debug(f"length of embedding {len(embedding)}, {embedding[:5]}")
    return embedding

In [2]:
vec = create_embedding_bge('test sentence')

In [7]:
import requests

# 调用生成列表的服务
question = '可以根据需要修改question参数'  # 可以根据需要修改question参数
response = requests.post('http://127.0.0.1:5111/genvec', json={'question': question})
vec = create_embedding_bge(question)
if response.status_code == 200:
    float_list = response.json()
    # 假设 float_list 和 vec 是之前获取的列表
# 检查类型是否都是 list
    if isinstance(float_list, list) and isinstance(vec, list):
        # 检查长度是否相同
        if len(float_list) == len(vec):
            # 逐个比较元素是否一致
            if all(a == b for a, b in zip(float_list, vec)):
                mylogger.debug("float_list 和 vec 相同")
            else:
                mylogger.error("float_list 和 vec 不同")
        else:
            mylogger.error("float_list 和 vec 长度不同")
    else:
        mylogger.error("float_list 和 vec 不是列表类型")
        # 判断列表长度
        mylogger.debug(f"Length of the list: {len(float_list)}")    
    
else:
    mylogger.error("Failed to retrieve the list.")

2024-03-22 23:14:21.290 | DEBUG    | __main__:<module>:16 - float_list 和 vec 相同


In [3]:
import mysql.connector
mysql_db_config = {
    "host": "10.101.9.50",
    "user": 'root',
    "password": 'newpass',
    "database": 'codebase'
}

db_connection = mysql.connector.connect(**mysql_db_config)
db_cursor = db_connection.cursor()
if db_connection.is_connected():
    mylogger.debug("Successfully connect mysql database")

mylogger.debug(mysql_db_config)
def execute_query(query, params=()):
    db_cursor.execute(query, params)
    if query.strip().startswith("SELECT"):
        pass
    else:
        db_connection.commit() 

2024-03-23 00:17:17.569 | DEBUG    | __main__:<module>:12 - Successfully connect mysql database
2024-03-23 00:17:17.571 | DEBUG    | __main__:<module>:14 - {'host': '10.101.9.50', 'user': 'root', 'password': 'newpass', 'database': 'codebase'}


In [4]:
query = 'SELECT username, last_refresh FROM user_sessions WHERE userKey = %s'
execute_query(query, ('f3082fc9-bc97-4d91-b600-b860e7cf609c',))
res = db_cursor.fetchone()
if res:
    print(1)
else:
    print(0)

1


In [18]:
type(res)

NoneType

In [13]:
if res:
    print(1)